In [30]:


import sys
import os
import hashlib
sys.path.append(os.path.abspath(os.path.join('..')))
from utils.simple_metrics import compute_and_store_metrics
from utils.visualization import compare_groundtruth_vs_corrected_histograms, compare_metrics_versus_bug_histograms, plot_column_distribution, plot_metrics_boxplots
from bug_detective.detective import process_prompt_dataset
from data_loader.loader import load_buggy_dataset, load_data, save_data
from utils.data_utils import tidy_responses, transform_to_tidy
import numpy as np
import pandas as pd

### System prompt experiment


In [31]:
baseline = load_data('300_metrics_prompt_baseline_system_none.pkl')
system_apr = load_data('300_metrics_prompt_baseline_system_apr.pkl')

In [32]:
def create_identifier(df: pd.DataFrame, prompt: str) -> pd.DataFrame:
    df['sample_uuid'] = df.apply(
        lambda row: hashlib.md5(
            (row['before_merge_without_docstrings'] + row['after_merge_without_docstrings']).encode('utf-8')
        ).hexdigest(),
        axis=1
    )
    df['prompt'] = prompt

    return df
baseline = create_identifier(baseline,"baseline")
system_apr = create_identifier(system_apr,"system_apr")
baseline[baseline['sample_uuid']=='8caf7a732dd1acbd2731a012a8bd45fe'].head()

,before_merge_without_docstrings,after_merge_without_docstrings,traceback_type,full_traceback,response_qwen-2.5-coder-32b-instruct,response_codestral-2501,response_gpt-4o,response_claude-3.5-sonnet,metric_qwen-2.5-coder-32b-instruct_ast_score,metric_qwen-2.5-coder-32b-instruct_text_score,...,metric_claude-3.5-sonnet_ast_score,metric_claude-3.5-sonnet_text_score,metric_claude-3.5-sonnet_ast_score_normalized,metric_claude-3.5-sonnet_codebleu,metric_claude-3.5-sonnet_ngram_match_score,metric_claude-3.5-sonnet_weighted_ngram_match_score,metric_claude-3.5-sonnet_syntax_match_score,metric_claude-3.5-sonnet_dataflow_match_score,sample_uuid,prompt
75748,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...",IndexError,An unexpected error has occurred: IndexError: ...,"def run(self, cmd, **kwargs):\n self._creat...","def run(self, cmd, **kwargs):\n self._creat...","def run(self, cmd, **kwargs):\n self._creat...","def run(self, cmd, **kwargs):\n self._creat...",0.0,0.688742,...,0.0,0.498246,0.0,0.484122,0.294467,0.33963,0.6,0.647059,8caf7a732dd1acbd2731a012a8bd45fe,baseline


In [33]:
responses  = pd.concat([tidy_responses(baseline),tidy_responses(system_apr)],ignore_index=True).reset_index(drop=True)
responses.head()

,model_name,sample_uuid,prompt,before_merge_without_docstrings,after_merge_without_docstrings,llm_response
0,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
1,qwen-2.5-coder-32b-instruct,d1e8f77aa937f586008b06aabb60d1ac,baseline,"def process(self, argv=None, executable=No...","def process(self, argv=None, executable=No...","def process(self, argv=None, executable=None, ..."
2,qwen-2.5-coder-32b-instruct,1ca9ee95a397f49814adbe7531b4e88a,baseline,"def backprojection(calibrated_event_list, pixe...","def backprojection(calibrated_event_list, pixe...","def backprojection(calibrated_event_list, pixe..."
3,qwen-2.5-coder-32b-instruct,2b0af7a3b93947509281e74e469190c6,baseline,"def export_annotations(self,export_range,e...","def export_annotations(self,export_range,e...","def export_annotations(self, export_range, exp..."
4,qwen-2.5-coder-32b-instruct,9fcbb3b0aa8818b51aba3778799ed601,baseline,"def summarize_corpus(corpus, ratio=0.2):\n ...","def summarize_corpus(corpus, ratio=0.2):\n ...","def summarize_corpus(corpus, ratio=0.2):\n ..."


In [34]:
responses[responses['sample_uuid'] == '8caf7a732dd1acbd2731a012a8bd45fe']

,model_name,sample_uuid,prompt,before_merge_without_docstrings,after_merge_without_docstrings,llm_response
0,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
300,codestral-2501,8caf7a732dd1acbd2731a012a8bd45fe,baseline,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
600,gpt-4o,8caf7a732dd1acbd2731a012a8bd45fe,baseline,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
900,claude-3.5-sonnet,8caf7a732dd1acbd2731a012a8bd45fe,baseline,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
1200,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,system_apr,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
1500,codestral-2501,8caf7a732dd1acbd2731a012a8bd45fe,system_apr,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
1800,gpt-4o,8caf7a732dd1acbd2731a012a8bd45fe,system_apr,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."
2100,claude-3.5-sonnet,8caf7a732dd1acbd2731a012a8bd45fe,system_apr,"def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n sel...","def run(self, cmd, **kwargs):\n self._creat..."


In [35]:
metrics = pd.concat([transform_to_tidy(baseline),transform_to_tidy(system_apr)],ignore_index=True).reset_index(drop=True)
metrics[metrics['sample_uuid']=='8caf7a732dd1acbd2731a012a8bd45fe'].head(20)

,model_name,sample_uuid,prompt,traceback_type,metric_name,metric_value
0,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,ast_score,0.000000
300,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,text_score,0.688742
600,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,ast_score_normalized,0.000000
900,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,codebleu,0.533997
1200,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,ngram_match_score,0.422525
1500,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,weighted_ngram_match_score,0.432303
1800,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,syntax_match_score,0.600000
2100,qwen-2.5-coder-32b-instruct,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,dataflow_match_score,0.647059
2400,codestral-2501,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,ast_score,0.000000
2700,codestral-2501,8caf7a732dd1acbd2731a012a8bd45fe,baseline,IndexError,text_score,0.826667


In [36]:
grouped_stats = (
    metrics.groupby(['model_name', 'prompt', 'metric_name'])['metric_value']
    .agg(['mean', 'var'])  # ou 'std' se quiser o desvio padrão
    .reset_index()
    .rename(columns={'mean': 'metric_mean', 'var': 'metric_variance'})
).sort_values(['model_name','metric_name'])

grouped_stats.head(30)

,model_name,prompt,metric_name,metric_mean,metric_variance
0,claude-3.5-sonnet,baseline,ast_score,0.216925,0.127229
8,claude-3.5-sonnet,system_apr,ast_score,0.225564,0.133064
1,claude-3.5-sonnet,baseline,ast_score_normalized,0.287190,0.162709
9,claude-3.5-sonnet,system_apr,ast_score_normalized,0.377884,0.195984
2,claude-3.5-sonnet,baseline,codebleu,0.725317,0.030521
10,claude-3.5-sonnet,system_apr,codebleu,0.728701,0.030227
3,claude-3.5-sonnet,baseline,dataflow_match_score,0.705789,0.068100
11,claude-3.5-sonnet,system_apr,dataflow_match_score,0.717925,0.067390
4,claude-3.5-sonnet,baseline,ngram_match_score,0.635746,0.071476
12,claude-3.5-sonnet,system_apr,ngram_match_score,0.648448,0.070013


In [41]:
df = grouped_stats[grouped_stats['prompt'].isin(['baseline', 'system_apr'])].copy()

def compute_pct_gain(group):
    if set(group['prompt']) != {'baseline', 'system_apr'}:
        return pd.Series({'pct_gain': None})  # ignora grupos incompletos
    
    baseline = group[group['prompt'] == 'baseline']['metric_mean'].values[0]
    system = group[group['prompt'] == 'system_apr']['metric_mean'].values[0]
    
    pct_gain = ((system - baseline) / baseline * 100) if baseline != 0 else None
    pct_gain = round(pct_gain, 2) if pct_gain is not None else None
    return pd.Series({'pct_gain': pct_gain})

pct_gain_df = df.groupby(['model_name', 'metric_name']).apply(compute_pct_gain).reset_index()

pct_gain_df.head(50)

,model_name,metric_name,pct_gain
0,claude-3.5-sonnet,ast_score,3.98
1,claude-3.5-sonnet,ast_score_normalized,31.58
2,claude-3.5-sonnet,codebleu,0.47
3,claude-3.5-sonnet,dataflow_match_score,1.72
4,claude-3.5-sonnet,ngram_match_score,2.00
5,claude-3.5-sonnet,syntax_match_score,0.07
6,claude-3.5-sonnet,text_score,3.77
7,claude-3.5-sonnet,weighted_ngram_match_score,-0.05
8,codestral-2501,ast_score,-0.16
9,codestral-2501,ast_score_normalized,0.14
